In [1]:
#!pip install pyspark

In [2]:
""" !wget -qnc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -n -q ngrok-stable-linux-amd64.zip """

' !wget -qnc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip\n!unzip -n -q ngrok-stable-linux-amd64.zip '

In [3]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder                  
      .config('spark.ui.port', '4050')
      .appName("SparkUI Introdução")
      .getOrCreate()
)

23/10/15 17:32:41 WARN Utils: Your hostname, valdir-G5-5590 resolves to a loopback address: 127.0.1.1; using 192.168.0.54 instead (on interface enp3s0)
23/10/15 17:32:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/15 17:32:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
!./ngrok authtoken 2Wlm9bDY0eka79Pd7tED4KnBUZv_24dzLdjRdSZXTLHaKUxpS
get_ipython().system_raw('./ngrok http 4050 &')
!sleep 10
!curl -s http://localhost:4040/api/tunnels | grep -Po 'public_url":"(?=https)\K[^"]*'

/bin/bash: line 1: ./ngrok: No such file or directory


/bin/bash: line 1: ./ngrok: No such file or directory


In [5]:
# {
#     "id_transacao": 1000, 
#     "valor": "58931.97",
#     "remetente": {"nome": "Jonathan Gonsalves", "banco": "BTG", "tipo": "PF"}, 
#     "destinatario": {"nome": "Emanuella Moura", "banco": "Itau", "tipo": "PJ"}, 
#     "transaction_date": "2021-06-02", 
#     "chave_pix": "aleatoria", 
#     "fraude": "1"
# }

from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType, TimestampType

schema_remetente_destinatario = StructType([
    StructField('nome', StringType()),
    StructField('banco', StringType()),
    StructField('tipo', StringType()),
])


schema_base_pix = StructType([
    StructField('id_transacao', IntegerType()),
    StructField('valor', DoubleType()),
    StructField('remetente', schema_remetente_destinatario),
    StructField('destinatario', schema_remetente_destinatario),
    StructField('transaction_date', TimestampType()),
    StructField('chave_pix', StringType()),
    StructField('fraude', IntegerType())
])


caminho_json = './pix_transactions.json'

df = spark.read.json(
    caminho_json,
    schema=schema_base_pix,
    timestampFormat="yyyy-MM-dd"
)

In [6]:
df.show()

+------------+--------+--------------------+--------------------+-------------------+---------+------+
|id_transacao|   valor|           remetente|        destinatario|   transaction_date|chave_pix|fraude|
+------------+--------+--------------------+--------------------+-------------------+---------+------+
|        1000|    7.05|{Jonathan Gonsalv...|{Gabriel Cunha, I...|2022-03-19 00:00:00|      cpf|     0|
|        1001|   37.28|{Jonathan Gonsalv...|{Diego Souza, XP,...|2021-01-26 00:00:00|aleatoria|     0|
|        1002|  282.73|{Jonathan Gonsalv...|{Nicole Nunes, BT...|2022-05-31 00:00:00|aleatoria|     0|
|        1003| 8447.92|{Jonathan Gonsalv...|{Maria Fernanda C...|2022-07-04 00:00:00|aleatoria|     0|
|        1004|   58.51|{Jonathan Gonsalv...|{Isabel Silva, C6...|2021-09-11 00:00:00|aleatoria|     0|
|        1005| 6655.12|{Jonathan Gonsalv...|{Anthony Carvalho...|2022-02-11 00:00:00|  celular|     0|
|        1006| 9912.25|{Jonathan Gonsalv...|{Eloah Monteiro, ...|2022-05-

In [7]:
df.select('destinatario.nome').show()

+--------------------+
|                nome|
+--------------------+
|       Gabriel Cunha|
|         Diego Souza|
|        Nicole Nunes|
|Maria Fernanda Ca...|
|        Isabel Silva|
|    Anthony Carvalho|
|      Eloah Monteiro|
|        Sophie Rocha|
|      Pietro Ribeiro|
|      Eloah Teixeira|
|     Emanuella Sales|
|    Valentina Campos|
|       Stella Araujo|
|     Benicio Costela|
|      Joao Fernandes|
|   Gabriela da Rocha|
|      Larissa Aragao|
|           Theo Dias|
|        Danilo Jesus|
|       Bruno Correia|
+--------------------+
only showing top 20 rows



In [8]:
df.select('remetente.nome', 'destinatario.nome').show()

+------------------+--------------------+
|              nome|                nome|
+------------------+--------------------+
|Jonathan Gonsalves|       Gabriel Cunha|
|Jonathan Gonsalves|         Diego Souza|
|Jonathan Gonsalves|        Nicole Nunes|
|Jonathan Gonsalves|Maria Fernanda Ca...|
|Jonathan Gonsalves|        Isabel Silva|
|Jonathan Gonsalves|    Anthony Carvalho|
|Jonathan Gonsalves|      Eloah Monteiro|
|Jonathan Gonsalves|        Sophie Rocha|
|Jonathan Gonsalves|      Pietro Ribeiro|
|Jonathan Gonsalves|      Eloah Teixeira|
|Jonathan Gonsalves|     Emanuella Sales|
|Jonathan Gonsalves|    Valentina Campos|
|Jonathan Gonsalves|       Stella Araujo|
|Jonathan Gonsalves|     Benicio Costela|
|Jonathan Gonsalves|      Joao Fernandes|
|Jonathan Gonsalves|   Gabriela da Rocha|
|Jonathan Gonsalves|      Larissa Aragao|
|Jonathan Gonsalves|           Theo Dias|
|Jonathan Gonsalves|        Danilo Jesus|
|Jonathan Gonsalves|       Bruno Correia|
+------------------+--------------

In [9]:
df.write.mode('overwrite').partitionBy('chave_pix').parquet('outpute/pix')

# SparkSQL

In [10]:
spark.read.json(
    caminho_json,
    schema=schema_base_pix,
    timestampFormat="yyyy-MM-dd"
).createOrReplaceTempView('transacao_pix')

In [74]:
df2 = spark.read.json(
    caminho_json,
    schema=schema_base_pix,
    timestampFormat="yyyy-MM-dd"
).createOrReplaceTempView('transacao_pix')

In [11]:
spark.sql("select valor, remetente.nome from transacao_pix").show()

+--------+------------------+
|   valor|              nome|
+--------+------------------+
|    7.05|Jonathan Gonsalves|
|   37.28|Jonathan Gonsalves|
|  282.73|Jonathan Gonsalves|
| 8447.92|Jonathan Gonsalves|
|   58.51|Jonathan Gonsalves|
| 6655.12|Jonathan Gonsalves|
| 9912.25|Jonathan Gonsalves|
| 8212.91|Jonathan Gonsalves|
|   91.71|Jonathan Gonsalves|
|   44.29|Jonathan Gonsalves|
|     8.2|Jonathan Gonsalves|
|    1.69|Jonathan Gonsalves|
|  442.76|Jonathan Gonsalves|
|    5.02|Jonathan Gonsalves|
|79576.33|Jonathan Gonsalves|
|   96.61|Jonathan Gonsalves|
|   16.92|Jonathan Gonsalves|
| 7128.87|Jonathan Gonsalves|
| 1658.63|Jonathan Gonsalves|
| 3935.92|Jonathan Gonsalves|
+--------+------------------+
only showing top 20 rows



In [12]:
spark.sql("""

    select chave_pix, sum(valor) from 
          transacao_pix
          group by 1
          order by 2 desc

""").show()

+---------+--------------------+
|chave_pix|          sum(valor)|
+---------+--------------------+
|aleatoria| 3.059806293100002E8|
|  celular| 3.018847117200015E8|
|      cpf| 3.007901403699987E8|
|    email|2.9592901058000124E8|
+---------+--------------------+



In [14]:
# 1. Descubra a quantidade total de transações na base de dados.
spark.sql("select count(id_transacao) from transacao_pix").show()


+-------------------+
|count(id_transacao)|
+-------------------+
|             100000|
+-------------------+



In [15]:
# 2. Qual foi a quantia total de dinheiro transacional em cada ano?

spark.sql("""
    select year(transaction_date) as ano, sum(valor) from transacao_pix
    group by 1 """
).show()

+----+-------------------+
| ano|         sum(valor)|
+----+-------------------+
|2023|2.100659429999998E7|
|2022|6.092498168899953E8|
|2021|5.743280807899972E8|
+----+-------------------+



In [17]:
# 3. Descubra qual é a média, mínima e máxima do valor de todas as transações.

spark.sql(""" 
    select avg(valor), max(valor), min(valor)
          from transacao_pix
""").show()

+-----------------+----------+----------+
|       avg(valor)|max(valor)|min(valor)|
+-----------------+----------+----------+
|12045.84491979997|  99999.54|       0.0|
+-----------------+----------+----------+



In [31]:
# 4. Quais foram as 5 maiores transações durante todo o período?
spark.sql("""
          select valor as 5_maiores_transacoes, remetente.banco
          from  transacao_pix order by valor desc limit 5
""").show()

+--------------------+-----+
|5_maiores_transacoes|banco|
+--------------------+-----+
|            99999.54|  BTG|
|            99980.03|  BTG|
|            99969.06|  BTG|
|            99964.99|  BTG|
|            99961.28|  BTG|
+--------------------+-----+



In [39]:
# 5. Para o banco BTG, mostre qual é a chave pix mais utilizada para enviar ou receber transações. 
spark.sql("""
    select chave_pix, count(chave_pix) as quantidade
          from transacao_pix
          where remetente.banco = 'BTG'
           group by 1
          order by 2 desc
          limit 1
          
""").show()

+---------+----------+
|chave_pix|quantidade|
+---------+----------+
|      cpf|     25179|
+---------+----------+



In [54]:
# 6. Descubra qual foi a maior transação de cada mês na base de dados.
spark.sql("""
    select  max(valor), month(transaction_date) as mes
          from transacao_pix
          where year(transaction_date) = 2021
          group by 2
            order by 2
""").show()

spark.sql("""
    select  max(valor), month(transaction_date) as mes
          from transacao_pix
          where year(transaction_date) = 2022
          group by 2
            order by 2
""").show()

spark.sql("""
    select  max(valor), month(transaction_date) as mes
          from transacao_pix
          where year(transaction_date) = 2023
          group by 2
            order by 2
""").show()

+----------+---+
|max(valor)|mes|
+----------+---+
|  99875.77|  1|
|  99822.15|  2|
|  99764.16|  3|
|  99795.86|  4|
|  99964.99|  5|
|  99543.53|  6|
|  99946.78|  7|
|  99813.42|  8|
|  99969.06|  9|
|  99810.18| 10|
|  99533.55| 11|
|  99999.54| 12|
+----------+---+



+----------+---+
|max(valor)|mes|
+----------+---+
|  99782.09|  1|
|  99842.28|  2|
|  99933.09|  3|
|  99903.29|  4|
|  99951.02|  5|
|  99949.36|  6|
|  99866.99|  7|
|  99980.03|  8|
|  99961.28|  9|
|  99724.27| 10|
|  99935.45| 11|
|  99941.77| 12|
+----------+---+



+----------+---+
|max(valor)|mes|
+----------+---+
|  99269.24|  1|
+----------+---+



In [56]:
# 7. Mostre a quantidade de transações que foram classificadas como fraude por ano.
from ipywidgets import interact
@interact(ano=[2021, 2022, 2023])
def show_max(ano):
    spark.sql(f"""
        select  max(valor), month(transaction_date) as mes
          from transacao_pix
          where year(transaction_date) = {ano}
          group by 2
            order by 2
    """).show()



interactive(children=(Dropdown(description='ano', options=(2021, 2022, 2023), value=2021), Output()), _dom_cla…

In [ ]:
# 7. Crie uma nova coluna que contenha o valor da transação em dólar. 

In [64]:
# 8. Descubra a quantidade de dinheiro que entrou na Nubank em todo o período.
spark.sql("""
    select sum(valor) from transacao_pix
      where destinatario.banco = 'Nubank' and year(transaction_date) between 2021 and 2023
""").show()

+-------------------+
|         sum(valor)|
+-------------------+
|1.687977762800004E8|
+-------------------+



In [78]:
df2 = spark.sql("select * from transacao_pix")

In [79]:
df2.show()

+------------+--------+--------------------+--------------------+-------------------+---------+------+
|id_transacao|   valor|           remetente|        destinatario|   transaction_date|chave_pix|fraude|
+------------+--------+--------------------+--------------------+-------------------+---------+------+
|        1000|    7.05|{Jonathan Gonsalv...|{Gabriel Cunha, I...|2022-03-19 00:00:00|      cpf|     0|
|        1001|   37.28|{Jonathan Gonsalv...|{Diego Souza, XP,...|2021-01-26 00:00:00|aleatoria|     0|
|        1002|  282.73|{Jonathan Gonsalv...|{Nicole Nunes, BT...|2022-05-31 00:00:00|aleatoria|     0|
|        1003| 8447.92|{Jonathan Gonsalv...|{Maria Fernanda C...|2022-07-04 00:00:00|aleatoria|     0|
|        1004|   58.51|{Jonathan Gonsalv...|{Isabel Silva, C6...|2021-09-11 00:00:00|aleatoria|     0|
|        1005| 6655.12|{Jonathan Gonsalv...|{Anthony Carvalho...|2022-02-11 00:00:00|  celular|     0|
|        1006| 9912.25|{Jonathan Gonsalv...|{Eloah Monteiro, ...|2022-05-

In [87]:
df2.filter(df2['destinatario.banco'] == 'Nubank').groupBy('destinatario.banco').sum('valor').show()

+------+-------------------+
| banco|         sum(valor)|
+------+-------------------+
|Nubank|1.687977762800004E8|
+------+-------------------+

